In [1]:
from edgar_functions_V2 import *
import numpy as np

ticker = "AMD"

In [2]:
scorecard_order = [
    "Net Revenue",
    "Cost of Revenue",
    "Gross Profit",
    "Gross Margin %",
    "SG&A",
    "Operating Income",
    "Operating Margin %",
    "Interest Expense",
    "EBT",
    "Tax Expense",
    "Tax Rate %",
    "Net Income",
    "EPS",
    "EPS Diluted",
    "EBITDA",
    "EBITDA Margin %",
    "Dividend",
    "Dividend Payout Ratio %",
    "Cash",
    "Marketable Securities",
    "Inventory",
    "Accounts Receivable",
    "Other Current Assets",
    "Total Current Assets",
    "Property, Plant, and Eqipment",
    "Right of Use Assets",
    "Other Long Term Assets",
    "Goodwill",
    "Intangible Assets",
    "Total Assets",
    "Accounts Payable",
    "Accrued Expenses",
    "Deferred Revenue",
    "Income Tax Payable",
    "Operating Lease Liability",
    "Other Current Liabilites",
    "Total Current Liabilities",
    "Long Term Debt",
    "Other Long Term Liabilities",
    "Total Liabilities",
    "Additional Paid In Capital",
    "Retained Earnings",
    "Total Stockholders Equity",
    "Total Liabilities and Stockholders Equity",
    "Shares Outstanding (Diluted)",
    "Book Value Per Share",
    "Operating Cash Flow",
    "OCF/NI",
    "Depreciation and Amortization",
    "Capex",
    "Capex / Depreciation",
    "Free Cash Flow",
    "Dividend Payment",
    "Stock Repurchase",
    "ROA %",
    "ROE %",
    "Profit Margin %",
    "Asset Turnover",
    "Current Ratio",
    "Equity Multiplier",
    "Net Working Capital",
    "Debt to Equity Ratio",
    "Debt to Assets Ratio",
    "Days Sales Outstanding",
    "Days of Inventory on Hand",
    "Payables Period",
    "Receivables Turnover",
    "Cash Conversion Cycle",
]

In [3]:
def combine_or_add_columns(df, cols, add_instead=False, replace_zero=False):
    if not cols:
        return None

    primary_col = None

    for col in cols:
        if col in df.columns:
            primary_col = df[col]
            break

    if primary_col is None:
        return None

    if len(cols) == 1:
        return primary_col

    for col in cols[1:]:
        if col in df.columns:
            # Skip if the column contains all NaN values
            if df[col].isna().all():
                continue

            if add_instead:
                primary_col = primary_col.add(df[col], fill_value=0).combine_first(
                    primary_col.combine_first(df[col])
                )
            else:
                if replace_zero:
                    primary_col = primary_col.mask(primary_col == 0).combine_first(
                        df[col]
                    )
                else:
                    primary_col = primary_col.combine_first(df[col])

    return primary_col

In [4]:
annual_df = process_and_aggregate_annual_data(ticker)
income_mapping = income_statement_mapping.income_mapping
balance_mapping = balance_sheet_mapping.balance_mapping
cash_mapping = cash_flow_mapping.cash_mapping
income = get_changed_columns(income_mapping, annual_df).droplevel(1, axis=1)
balance = get_changed_columns(balance_mapping, annual_df).droplevel(1, axis=1)
cash_flow = get_changed_columns(cash_mapping, annual_df).droplevel(1, axis=1)

income_df = income.T.groupby(income.columns).mean().sort_index(ascending=False)
income_df = income_df.sort_index(axis=1, ascending=False)
income_df = income_df.T
income_df["Cost of Revenue"] = income_df["Net Revenue"] - income_df["Gross Profit"]
income_df["EBITDA"] = (
    income_df["EBT"]
    + income_df["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)

balance_df = balance.T.groupby(balance.columns).mean().sort_index(ascending=False)
balance_df = balance_df.T.sort_index(axis=1, ascending=False)
balance_df["Total Equity(calc)"] = (
    balance_df["Total Assets"] - balance_df["Total Liabilities"]
)
balance_df["Toal Liabilites and Equity(calc)"] = balance_df[
    "Total Liabilities"
] + combine_or_add_columns(balance_df, ["Total Equity", "Total Equity(calc)"])
cash_df = cash_flow.T.groupby(cash_flow.columns).mean().sort_index(ascending=False)
cash_df = cash_df.T.sort_index(axis=1, ascending=False)
cash_df["Depreciation and Amortization(calc)"] = combine_or_add_columns(
    cash_df, ["Depreciation", "Amortization"], add_instead=True
)

KeyError: 'Total Liabilities'

In [ ]:
scorecard = pd.DataFrame()
scorecard["Net Revenue"] = combine_or_add_columns(income_df, ["Net Revenue"])
scorecard["Cost of Revenue"] = combine_or_add_columns(income_df, ["Cost of Revenue"])
scorecard["Gross Profit"] = combine_or_add_columns(income_df, ["Gross Profit"])
scorecard["Gross Margin %"] = scorecard["Gross Profit"] / scorecard["Net Revenue"] * 100
scorecard["SG&A"] = combine_or_add_columns(income_df, ["SG&A"])
scorecard["Operating Income"] = combine_or_add_columns(income_df, ["Operating Income"])
scorecard["Operating Margin %"] = (
    scorecard["Operating Income"] / scorecard["Net Revenue"]
) * 100
scorecard["Interest Expense"] = combine_or_add_columns(income_df, ["Interest Expense"])
scorecard["EBT"] = combine_or_add_columns(
    income_df, ["EBT", "EBT and equity investments"]
)
scorecard["Tax Expense"] = combine_or_add_columns(income_df, ["Taxes"])
scorecard["Tax Rate %"] = combine_or_add_columns(
    income_df, ["Tax Rate Continuing Operations"]
)
scorecard["Tax Rate %"] = scorecard["Tax Rate %"] * 100
scorecard["Net Income"] = combine_or_add_columns(income_df, ["Net Income"])
scorecard["EPS"] = combine_or_add_columns(income_df, ["EPS"])
scorecard["EPS Diluted"] = combine_or_add_columns(income_df, ["EPS Diluted"])
scorecard["EBITDA"] = (
    scorecard["EBT"]
    + scorecard["Interest Expense"].notna()
    + income_df["Depreciation and Amortization"]
)
scorecard["EBITDA Margin %"] = (scorecard["EBITDA"] / scorecard["Net Revenue"]) * 100


scorecard["Cash"] = combine_or_add_columns(
    balance_df, ["Cash", "Cash and Restricted Cash"]
)
scorecard["Marketable Securities"] = combine_or_add_columns(
    balance_df, ["Marketable Securities"]
)
scorecard["Inventory"] = balance_df["Inventory"]
scorecard["Accounts Receivable"] = balance_df["Accounts Receivable"]
scorecard["Other Current Assets"] = balance_df["Other Current Assets"]
scorecard["Total Current Assets"] = balance_df["Total Current Assets"]
scorecard["Property, Plant, and Eqipment"] = balance_df[
    "Property, Plant, and Equipment"
]
scorecard["Right of Use Assets"] = combine_or_add_columns(
    balance_df, ["Operating Lease Right of Use Asset"]
)
scorecard["Other Long Term Assets"] = combine_or_add_columns(
    balance_df, ["Other Long Term Assets"]
)
scorecard["Goodwill"] = balance_df["Goodwill Asset"]
scorecard["Intangible Assets"] = combine_or_add_columns(
    balance_df, ["Intangible Assets"]
)
scorecard["Total Assets"] = balance_df["Total Assets"]
scorecard["Accounts Payable"] = combine_or_add_columns(balance_df, ["Accounts Payable"])
scorecard["Accrued Expenses"] = combine_or_add_columns(balance_df, ["Accrued Expenses"])
scorecard["Deferred Revenue"] = combine_or_add_columns(balance_df, ["Deferred Revenue"])
scorecard["Income Tax Payable"] = combine_or_add_columns(
    balance_df, ["Income Taxes Payable"]
)
scorecard["Operating Lease Liability"] = combine_or_add_columns(
    balance_df, ["Operating Lease Liability"]
)
scorecard["Other Current Liabilites"] = combine_or_add_columns(
    balance_df, ["Other Current Liabilities"]
)
scorecard["Total Current Liabilities"] = balance_df["Total Current Liabilities"]
scorecard["Long Term Debt"] = combine_or_add_columns(
    balance_df, ["Long Term Debt", "Long Term Lease Liability"], replace_zero=True
)
scorecard["Other Long Term Liabilities"] = combine_or_add_columns(
    balance_df, ["Other Long Term Liabilities"]
)
scorecard["Total Liabilities"] = combine_or_add_columns(
    balance_df, ["Total Liabilities"]
)
scorecard["Additional Paid In Capital"] = combine_or_add_columns(
    balance_df, ["Additional Paid in Capital"]
)
scorecard["Retained Earnings"] = combine_or_add_columns(
    balance_df, ["Retained Earnings"]
)
scorecard["Total Stockholders Equity"] = combine_or_add_columns(
    balance_df, ["Total Stockholders Equity", "Total Equity(calc)"]
)
scorecard["Total Liabilities and Stockholders Equity"] = combine_or_add_columns(
    balance_df,
    ["Total Liabilities and Stockholders Equity", "Toal Liabilites and Equity(calc)"],
)
scorecard["Shares Outstanding (Diluted)"] = combine_or_add_columns(
    income_df,
    ["Weighted Average Shares Diluted", "Weighted Average Shares Outstanding"],
    replace_zero=True,
)
scorecard["Book Value Per Share"] = (
    scorecard["Total Stockholders Equity"] / scorecard["Shares Outstanding (Diluted)"]
)
scorecard["Operating Cash Flow"] = combine_or_add_columns(
    cash_df, ["Net Cash Provided by Operating Activities"]
)
scorecard["OCF/NI"] = scorecard["Operating Cash Flow"] / scorecard["Net Income"]
scorecard["Depreciation and Amortization"] = combine_or_add_columns(
    cash_df, ["Depreciation and Amortization", "Depreciation and Amortization(calc)"]
)
scorecard["Capex"] = combine_or_add_columns(
    cash_df, ["Purchase of Property, Plant, and Equipment"]
)
scorecard["Capex"] = -scorecard["Capex"]
scorecard["Capex / Depreciation"] = (
    scorecard["Capex"] / scorecard["Depreciation and Amortization"]
)
scorecard["Free Cash Flow"] = scorecard["Operating Cash Flow"] + scorecard["Capex"]
scorecard["Dividend Payment"] = combine_or_add_columns(
    cash_df, ["Payment of Dividends"]
)
scorecard["Dividend Payment"] = -scorecard["Dividend Payment"]
scorecard["Stock Repurchase"] = combine_or_add_columns(
    cash_df, ["Repurchases of Common Stock"]
)
scorecard["Stock Repurchase"] = -scorecard["Stock Repurchase"]
scorecard["ROA %"] = (scorecard["Net Income"] / scorecard["Total Assets"]) * 100
scorecard["ROE %"] = (
    scorecard["Net Income"] / scorecard["Total Stockholders Equity"]
) * 100
scorecard["Profit Margin %"] = scorecard["Net Income"] / scorecard["Net Revenue"] * 100
scorecard["Asset Turnover"] = scorecard["Net Revenue"] / scorecard["Total Assets"]
scorecard["Current Ratio"] = (
    scorecard["Total Current Assets"] / scorecard["Total Current Liabilities"]
)
scorecard["Equity Multiplier"] = (
    scorecard["Total Assets"] / scorecard["Total Stockholders Equity"]
)
scorecard["Net Working Capital"] = (
    scorecard["Total Current Assets"] - scorecard["Total Current Liabilities"]
)
scorecard["Debt to Equity Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Stockholders Equity"]
)
scorecard["Debt to Assets Ratio"] = (
    scorecard["Total Liabilities"] / scorecard["Total Assets"]
)
scorecard["Days Sales Outstanding"] = (
    scorecard["Accounts Receivable"] / scorecard["Net Revenue"]
) * 365
scorecard["Days of Inventory on Hand"] = (
    scorecard["Inventory"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Payables Period"] = (
    scorecard["Accounts Payable"] / scorecard["Cost of Revenue"]
) * 365
scorecard["Receivables Turnover"] = (
    scorecard["Net Revenue"] / scorecard["Accounts Receivable"]
)
scorecard["Cash Conversion Cycle"] = (
    scorecard["Days Sales Outstanding"]
    + scorecard["Days of Inventory on Hand"]
    - scorecard["Payables Period"]
)
scorecard["Dividend"] = (
    -scorecard["Dividend Payment"] / scorecard["Shares Outstanding (Diluted)"]
)
scorecard["Dividend Payout Ratio %"] = (scorecard["Dividend"] / scorecard["EPS"]) * 100
scorecard = scorecard[scorecard_order]

In [ ]:
scorecard.T

,2022,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010,2009,2008,2007
Net Revenue,"394,328,000,000","365,817,000,000","274,515,000,000","260,174,000,000","265,595,000,000","229,234,000,000","215,639,000,000","233,715,000,000","182,795,000,000","170,910,000,000","156,508,000,000","108,249,000,000","65,225,000,000","42,905,000,000","37,491,000,000","24,578,000,000"
Cost of Revenue,"223,546,000,000","212,981,000,000","169,559,000,000","161,782,000,000","163,756,000,000","141,048,000,000","131,376,000,000","140,089,000,000","112,258,000,000","106,606,000,000","87,846,000,000","64,431,000,000","39,541,000,000","25,683,000,000","24,294,000,000","16,426,000,000"
Gross Profit,"170,782,000,000","152,836,000,000","104,956,000,000","98,392,000,000","101,839,000,000","88,186,000,000","84,263,000,000","93,626,000,000","70,537,000,000","64,304,000,000","68,662,000,000","43,818,000,000","25,684,000,000","17,222,000,000","13,197,000,000","8,152,000,000"
Gross Margin %,43.31,41.78,38.23,37.82,38.34,38.47,39.08,40.06,38.59,37.62,43.87,40.48,39.38,40.14,35.20,33.17
SG&A,"25,094,000,000","21,973,000,000","19,916,000,000","18,245,000,000","16,705,000,000","15,261,000,000","14,194,000,000","14,329,000,000","11,993,000,000","10,830,000,000","10,040,000,000","7,599,000,000","5,517,000,000","4,149,000,000","3,761,000,000","2,963,000,000"
Operating Income,"119,437,000,000","108,949,000,000","66,288,000,000","63,930,000,000","70,898,000,000","61,344,000,000","60,024,000,000","71,230,000,000","52,503,000,000","48,999,000,000","55,241,000,000","33,790,000,000","18,385,000,000","11,740,000,000","8,327,000,000","4,407,000,000"
Operating Margin %,30.29,29.78,24.15,24.57,26.69,26.76,27.84,30.48,28.72,28.67,35.30,31.22,28.19,27.36,22.21,17.93
Interest Expense,"2,931,000,000","2,645,000,000","2,873,000,000","3,576,000,000","3,240,000,000","2,323,000,000","1,456,000,000","733,000,000","384,000,000","136,000,000",0,0,NaN,NaN,NaN,NaN
EBT,"119,103,000,000","109,207,000,000","67,091,000,000","65,737,000,000","72,903,000,000","64,089,000,000","61,372,000,000","72,515,000,000","53,483,000,000","50,155,000,000","55,763,000,000","34,205,000,000","18,540,000,000","12,066,000,000","8,947,000,000","5,006,000,000"
Tax Expense,"19,300,000,000","14,527,000,000","9,680,000,000","10,481,000,000","13,372,000,000","15,738,000,000","15,685,000,000","19,121,000,000","13,973,000,000","13,118,000,000","14,030,000,000","8,283,000,000","4,527,000,000","3,831,000,000","2,828,000,000","1,511,000,000"
